In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

# Set seed
torch.manual_seed(0)
np.random.seed(0)

# Sample corpus
corpus = "deep learning is powerful and fun and powerful models are effective in learning"
words = corpus.split()
vocab = sorted(set(words))
word_to_idx = {w: i for i, w in enumerate(vocab)}
idx_to_word = {i: w for w, i in word_to_idx.items()}

# Hyperparameters
sequence_len = 3
embedding_dim = 10
hidden_dim = 32
vocab_size = len(vocab)

# Prepare data
def make_sequences(words):
    sequences = []
    for i in range(len(words) - sequence_len):
        seq = words[i:i+sequence_len]
        target = words[i+sequence_len]
        sequences.append(([word_to_idx[w] for w in seq], word_to_idx[target]))
    return sequences

sequences = make_sequences(words)

# Model
class GRUTextGen(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(GRUTextGen, self).__init__()
        self.embed = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden=None):
        x = self.embed(x)
        out, hidden = self.gru(x, hidden)
        out = self.fc(out[:, -1, :])  # Use last output
        return out, hidden

# Instantiate model
model = GRUTextGen(vocab_size, embedding_dim, hidden_dim)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training
for epoch in range(300):
    total_loss = 0
    for seq, target in sequences:
        seq_tensor = torch.tensor(seq).unsqueeze(0)
        target_tensor = torch.tensor([target])
        optimizer.zero_grad()
        output, _ = model(seq_tensor)
        loss = loss_fn(output, target_tensor)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    if epoch % 50 == 0 or epoch == 299:
        print(f"Epoch {epoch+1}/300, Loss: {total_loss:.4f}")

# Sampling
def sample_from_probs(probs, temperature=1.0):
    probs = torch.softmax(probs / temperature, dim=-1).detach().cpu().numpy()
    return np.random.choice(len(probs), p=probs)

def generate_text(model, seed_words, length=6, temperature=1.0):
    model.eval()
    generated = seed_words[:]
    hidden = None
    for _ in range(length):
        input_seq = [word_to_idx[w] for w in generated[-sequence_len:]]
        input_tensor = torch.tensor(input_seq).unsqueeze(0)
        output, hidden = model(input_tensor, hidden)
        next_idx = sample_from_probs(output[0], temperature)
        next_word = idx_to_word[next_idx]
        generated.append(next_word)
    return ' '.join(generated)

# Try it
seed = ['deep', 'learning', 'is']
print("\nGenerated Text (temp=1.0):")
print(generate_text(model, seed_words=seed, length=6, temperature=1.0))

print("\nGenerated Text (temp=0.5):")
print(generate_text(model, seed_words=seed, length=6, temperature=0.5))
